<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/01.%20Using_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-intro} -->

# Understanding LLM APIs

We will explore OpenAI models API to generate text.

<!--- @wandbcode{llmapps-intro} -->

### Setup

In [1]:
!pip install google-generativeai weave -qqU

In [2]:
import os
from getpass import getpass
import wandb
import google.generativeai as genai
import weave

In [3]:
os.environ["GOOGLE_API_KEY"] = getpass("Paste your Google API key: \n")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
genai.configure(api_key=GOOGLE_API_KEY)

Paste your Google API key: 
 ········


In [6]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Say something about Weights & Biases")
print(response.text)

Weights & Biases (W&B) is a powerful platform for **experiment tracking, model visualization, and collaboration in machine learning**. It simplifies the process of:

* **Tracking experiments:**  W&B lets you automatically log metrics, parameters, code, and other artifacts associated with your machine learning experiments. This provides a clear and organized history of your work, making it easy to compare different approaches and identify trends.

* **Visualizing models:**  W&B offers interactive visualizations for model performance, loss curves, feature importance, and other insights. This allows you to gain a deeper understanding of your models and identify potential areas for improvement.

* **Collaborating with teams:**  W&B facilitates collaboration by providing a shared workspace where team members can track experiments, view visualizations, and discuss results. This promotes transparency and helps teams iterate faster.

**Key Features of W&B:**

* **Experiment Tracking:**  Logs m

In [20]:
%pip install --upgrade openai==0.28 tiktoken wandb -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
weave 0.51.8 requires openai>=1.0.0, but you have openai 0.28.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import openai
import tiktoken
import wandb
from pprint import pprint
from getpass import getpass
from wandb.integration.openai import autolog

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [2]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Paste your OpenAI key from: https://platform.openai.com/account/api-keys
 ········


OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [3]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "introduction"})

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: supriyagdptl. Use `wandb login --relogin` to force relogin


TypeError: __name__ must be set to a string object

# Tokenization

In [9]:
encoding = tiktoken.encoding_for_model("davinci-002")
enc = encoding.encode("Weights & Biases is awesome!")
print(enc)
print(encoding.decode(enc))

[56730, 612, 12371, 2315, 374, 12738, 0]
Weights & Biases is awesome!


we can decode the tokens one by one

In [10]:
for token_id in enc:
    print(f"{token_id}\t{encoding.decode([token_id])}")

56730	Weights
612	 &
12371	 Bi
2315	ases
374	 is
12738	 awesome
0	!


> Note how the leading tokens contain spacing.

# Sampling

Let's sample some text from the model. For this, let's create a wrapper function around the temperature parameters.
Higher temperature will result in more random samples.

In [4]:
def generate_with_temperature(temp):
  "Generate text with a given temperature, higher temperature means more randomness"
  response = openai.Completion.create(
    model="davinci-002",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    temperature=temp,
  )
  return response.choices[0].text

for temp in [0]:#, 0.5, 1, 1.5, 2]:
  pprint(f'TEMP: {temp}, GENERATION: {generate_with_temperature(temp)}')

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


You can also use the [`top_p` parameter](https://platform.openai.com/docs/api-reference/completions/create#completions/create-top_p) to control the diversity of the generated text. This parameter controls the cumulative probability of the next token. For example, if `top_p=0.9`, the model will pick the next token from the top 90% most likely tokens. The higher the `top_p` the more likely the model will pick a token that it hasn't seen before. You should only use one of `temperature` or `top_p` at a given time.

In [19]:
openai migrate

SyntaxError: invalid syntax (2463506640.py, line 1)

In [ ]:
def generate_with_topp(topp):
  "Generate text with a given top-p, higher top-p means more randomness"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    top_p=topp,
    )
  return response.choices[0].text

In [ ]:
for topp in [0.01, 0.1, 0.5, 1]:
  pprint(f'TOP_P: {topp}, GENERATION: {generate_with_topp(topp)}')

# Chat API

Let's switch to chat mode and see how the model responds to our messages. We have some control over the model's response by passing a `system-role`, here we can steer to model to adhere to a certain behaviour.

> We are using `gpt-3.5-turbo`, this model is faster and cheaper than `davinci-003`

In [5]:
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say something about Weights & Biases"},
    ],
    temperature=0,
)

response

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


As you can see above, the response is a JSON object with relevant information about the request.

In [ ]:
pprint(response.choices[0].message.content)

In [ ]:
wandb.finish()